# About

This is just a small example of how we use the Spotify API to get all of our songs and rank them by popularity (i.e. a metric determined by Spotify). We can use this to figure out the "good" songs our friends listen to.

In [ ]:
import requests

In [ ]:
# spotify needs you to use a token using your client id and secret

def get_my_token(client_id, client_secret):
    endpoint = "https://accounts.spotify.com/api/token"
    data = {
        "grant_type": "client_credentials"
    }
    req = requests.post(endpoint, auth=(client_id, client_secret), data=data)
    return req.json()["access_token"]

In [ ]:
def search_user_playlists(user_id):
    endpoint = "https://api.spotify.com/v1/users/{}/playlists".format(user_id)
    headers = {"Authorization": "Bearer {}".format(access_token)}
    req = requests.get(endpoint, headers=headers)
    return req.json()

In [ ]:
def get_songs_from_playlist(playlist_id):
    endpoint = "https://api.spotify.com/v1/playlists/{}".format(playlist_id)
    headers = {"Authorization": "Bearer {}".format(access_token)}
    req = requests.get(endpoint, headers=headers)
    return req.json()

In [ ]:
import os

# just replace `os.environ["KEY"]` with your credentials (find this in your Spotify API dashboard)
CLIENT_ID = os.environ["CLIENT_ID"]
CLIENT_SECRET = os.environ["CLIENT_SECRET"]

access_token = get_my_token(CLIENT_ID, CLIENT_SECRET)

# just replace `os.environ["KEY"]` with any user id (find this in the URL by searching for a user in the Spotify web client)
USER_ID = os.environ["USER_ID"]
user_playlists = search_user_playlists(USER_ID)

In [ ]:
songs_collected = []

for playlist in user_playlists["items"]:
    songs = get_songs_from_playlist(playlist["id"])
    for song in songs["tracks"]["items"]:
        song_data = song["track"]
        
        try:
            song_extracted_data = {
                "id": song_data["id"],
                "name": song_data["name"],
                "artists": [artist["name"] for artist in song_data["artists"]],
                "popularity": song_data["popularity"],
                "preview": song_data["preview_url"]
            }
            songs_collected.append(song_extracted_data)
        except:
            pass

In [ ]:
# we will remove duplicate songs

seen = set()
unique_songs_collected = []

for song in songs_collected:
    if song["id"] in seen:
        continue
    unique_songs_collected.append(song)
    seen.add(song["id"])

In [ ]:
# we will print the songs in increasing popularity
for song in sorted(unique_songs_collected, key=lambda song: song["popularity"]):
    print(song["popularity"], song["name"], "--- by", song["artists"], song["preview"])

## Notes

Remember that a good developer should be reading more than they code, so please read the documentation! **Read Spotify API documentation here https://developer.spotify.com/documentation/**

We are not modifying user data (i.e. we can only read what is public), so we do not have full access to the Spotify API. We are only using the "Client Credentials flow" for app authorization. Read here https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow

If you want to modify user data, you can use the "Authorization Code flow" and set up the level of permissions (scope) you want to grant to your Python script. Read here https://developer.spotify.com/documentation/general/guides/authorization-guide/#authorization-code-flow


## Example projects:

- create a "songs popularity over time" API by logging song popularity every single day
- integrate this with the Genius API to identify song lyrics https://docs.genius.com/
- create a program that allows a user to duplicate another user's playlist (similar to how someone can fork a repo on Github)
- use unsupervised learning algorithms (possibly with the help of librosa) to cluster your friends based on their music tastes... maybe you will find out the kinds of people you like based on what they listen to :^)